In [1]:
import socket
import ssl
import json

def secure_socket_layer(server=True, port=65432):
    context = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH if server else ssl.Purpose.SERVER_AUTH)
    if server:
        context.load_cert_chain(certfile='server.crt', keyfile='server.key')  # Load server's certificate and key
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.bind(('localhost', port))
        server_socket.listen(5)
        secure_socket = context.wrap_socket(server_socket, server_side=True)
        return secure_socket
    else:
        context.load_verify_locations('server.crt')  # Load server's certificate for client verification
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        secure_socket = context.wrap_socket(client_socket, server_hostname='localhost')
        secure_socket.connect(('localhost', port))
        return secure_socket

def send_secure_message():
    ssock = secure_socket_layer(server=False)
    message = {
        "order_id": 12345,
        "customer_id": 67890,
        "payment_details": {
            "amount": 100,
            "currency": "USD"
        }
    }
    ssock.send(json.dumps(message).encode('utf-8'))
    print("Message and signature sent securely!")
    ssock.close()

def receive_secure_message():
    ssock = secure_socket_layer()
    client_socket, addr = ssock.accept()
    print(f"Connected securely by {addr}")
    data = client_socket.recv(1024)
    print("Received secure message:", data.decode('utf-8'))
    client_socket.close()
    ssock.close()

if __name__ == "__main__":
    # To run, start receive_secure_message() in one script and send_secure_message() in another
    pass
